# Main code

In [72]:
import requests
from pymongo import MongoClient
from pymongo import GEOSPHERE
import pandas as pd
import json
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import geopy
from geopy import distance 
import math

In [3]:
#1

In [4]:
def weighted_colleague(string):
    return (string/total_company)*100

In [8]:
designers = 20
ui_ux_engineers = 5
frontend_dev = 10
data_engineers = 15
backend_dev = 5
account_managers = 20
maint = 1
executives = 10
ceo = 1
total_company = 87

In [9]:
weighted_colleague(designers)

22.988505747126435

In [10]:
#2

In [36]:
def design_or_success_companies(database,coll_name):
    client = MongoClient("localhost:27017")
    db = client[(f"{database}")]
    c = db.get_collection(f"{coll_name}")
    
    condition_one = {"description": {"$regex": "design"}}
    condition_two = {"tag_list": {"$regex": "design"}}
    condition_three = {"name": {"$regex": "design"}}
    condition_four = {"category_code": {"$regex": "design"}}
    condition_five = {"total_money_raised": {"$regex": "M"}}
    condition_six = {"total_money_raised": {"$regex": "B"}}
    condition_seven = {}
    
    projection = {"name": 1, "number_of_employees": 1, "total_money_raised": 1,"offices.country_code": 1,"offices.city": 1, "_id":0}
    result_filtering = list(c.find({
    "$or": [
         condition_one,
          condition_two, condition_three,
            condition_four,
          condition_five,condition_six
    ], 
    "number_of_employees": {"$gte": 3000}
}, projection))
    return result_filtering

In [37]:
design_or_success_companies("Ironhack","companies")

[{'name': 'Facebook',
  'number_of_employees': 5299,
  'total_money_raised': '$2.43B',
  'offices': [{'city': 'Menlo Park', 'country_code': 'USA'},
   {'city': 'Dublin', 'country_code': 'IRL'},
   {'city': 'New York', 'country_code': 'USA'}]},
 {'name': 'eBay',
  'number_of_employees': 15000,
  'total_money_raised': '$6.7M',
  'offices': [{'city': 'San Jose', 'country_code': 'USA'}]},
 {'name': 'Cisco',
  'number_of_employees': 63000,
  'total_money_raised': '$2.5M',
  'offices': [{'city': 'San Jose', 'country_code': 'USA'}]},
 {'name': 'Yahoo!',
  'number_of_employees': 13600,
  'total_money_raised': '$6.8M',
  'offices': [{'city': 'Sunnyvale', 'country_code': 'USA'}]},
 {'name': 'Google',
  'number_of_employees': 28000,
  'total_money_raised': '$555M',
  'offices': [{'city': 'Mountain View', 'country_code': 'USA'},
   {'city': 'Ann Arbor', 'country_code': 'USA'},
   {'city': 'Atlanta', 'country_code': 'USA'},
   {'city': 'Austin', 'country_code': 'USA'},
   {'city': 'Boulder', 'count

In [47]:
#3

In [40]:
def calling_first_dataframe():
    df = pd.DataFrame(design_or_success_companies(database,coll_name))
    df["total_offices"] = df["offices"].apply(lambda x: len(x))
    df.sort_values(by="total_offices", ascending=False).reset_index().drop(['index'], axis=1)
    return df

In [45]:
#4

In [19]:
def dictionary_of_cities(df):
    
    city_dict = {}
    for item, rows in df.iterrows():
        for i in rows["offices"]:
            ciudad = i["city"]
            if ciudad in city_dict.keys():
                city_dict[ciudad] +=1
            else:
                city_dict[ciudad]  =1
    {k: v for k, v in sorted(city_dict.items(), key=lambda item: item[1])}
    return city_dict

In [44]:
#5

In [20]:
def dictionary_of_countries(df):
    country_dict = {}
    for item, rows in df.iterrows():
        for i in rows["offices"]:
            country = i["country_code"]
            if country in country_dict.keys():
                country_dict[country] +=1
            else:
                country_dict[country]  =1
    {k: v for k, v in sorted(country_dict.items(), key=lambda item: item[1])}
    country_dict

In [43]:
#6

In [48]:
def companies_latandlong(a,b):
    client = MongoClient("localhost:27017")
    db = client[f"{a}"]
    c = db.get_collection(f"{b}")
    
    condition_one = {"description": {"$regex": "design"}}
    condition_two = {"tag_list": {"$regex": "design"}}
    condition_three = {"name": {"$regex": "design"}}
    condition_four = {"category_code": {"$regex": "design"}}
    condition_five = {"total_money_raised": {"$regex": "M"}}
    condition_six = {"total_money_raised": {"$regex": "B"}}
    condition_seven = {}

    projection = {"name": 1, "number_of_employees": 1, "total_money_raised": 1,"offices.country_code": 1,"offices.city": 1,"offices.latitude": 1,"offices.longitude": 1, "_id":0}
    result_filtering_latlong = list(c.find({
        "$or": [
             condition_one,
              condition_two, condition_three,
                condition_four,
              condition_five,condition_six
        ], 
        "number_of_employees": {"$gte": 3000}
    }, projection))


    return result_filtering_latlong

In [45]:
companies_latandlong("Ironhack","companies")

{'name': 'Facebook',
 'number_of_employees': 5299,
 'total_money_raised': '$2.43B',
 'offices': [{'city': 'Menlo Park',
   'country_code': 'USA',
   'latitude': 37.41605,
   'longitude': -122.151801},
  {'city': 'Dublin',
   'country_code': 'IRL',
   'latitude': 53.344104,
   'longitude': -6.267494},
  {'city': 'New York',
   'country_code': 'USA',
   'latitude': 40.7557162,
   'longitude': -73.9792469}]}

In [42]:
#7

In [49]:
def companies_by_city(database,coll_name,city):
    client = MongoClient("localhost:27017")
    db = client[f"{database}"]
    c = db.get_collection(f"{coll_name}")
    
    condition_one = {"description": {"$regex": "design"}}
    condition_two = {"tag_list": {"$regex": "design"}}
    condition_three = {"name": {"$regex": "design"}}
    condition_four = {"category_code": {"$regex": "design"}}
    condition_five = {"total_money_raised": {"$regex": "M"}}
    condition_six = {"total_money_raised": {"$regex": "B"}}
    condition_seven = {"number_of_employees": {"$gte": 3000}}
    condition_eight = {"offices.city": {"$regex": f"{city}"}}
    
    projection = {"name": 1, "number_of_employees": 1, "total_money_raised": 1,"offices.country_code": 1,"offices.city": 1,"offices.latitude": 1,"offices.longitude": 1, "_id":0}
    result_filtering_latlong_nyc = list(c.find({
        "$and": [
            {
                "$or": [
                     condition_one,
                    condition_two, condition_three,
                    condition_four,
                      condition_five,condition_six
                        ]}, 

        condition_seven,condition_eight]}, projection))


    return result_filtering_latlong_nyc

In [52]:
companies_by_city("Ironhack","companies","Francisco")

[{'name': 'Google',
  'number_of_employees': 28000,
  'total_money_raised': '$555M',
  'offices': [{'city': 'Mountain View',
    'country_code': 'USA',
    'latitude': 37.421972,
    'longitude': -122.084143},
   {'city': 'Ann Arbor',
    'country_code': 'USA',
    'latitude': 42.280988,
    'longitude': -83.748882},
   {'city': 'Atlanta',
    'country_code': 'USA',
    'latitude': 33.781466,
    'longitude': -84.387519},
   {'city': 'Austin',
    'country_code': 'USA',
    'latitude': 30.351416,
    'longitude': -97.751382},
   {'city': 'Boulder',
    'country_code': 'USA',
    'latitude': 40.021416,
    'longitude': -105.260931},
   {'city': 'Cambridge',
    'country_code': 'USA',
    'latitude': 42.36305,
    'longitude': -71.087478},
   {'city': 'Chicago',
    'country_code': 'USA',
    'latitude': 41.889474,
    'longitude': -87.628912},
   {'city': 'Coppell',
    'country_code': 'USA',
    'latitude': 32.981465,
    'longitude': -97.018196},
   {'city': 'Dallas',
    'country_cod

In [ ]:
#8

In [54]:
def filtered_db_nyc(df,city):
    df2 = (df[df['city'] == f"{city}"]).reset_index().drop('index', axis=1) 
    df3 = df2[df2['latitude'].notna()]
    return df3

In [53]:
#9

In [55]:
def getting_coordinates_from_offices(df):
    list = []

    for item, rows in df.iterrows():
        for i in rows["offices"]:
            list.append(i)

    return list

In [56]:
#10

In [58]:
def dataframe_with_coordinates_by_city(list):
    table_with_coordinates = pd.DataFrame(list)
    return table_with_coordinates

In [59]:
#11

In [60]:
def my_empty_map(lat,lon):
    map  = Map(location = [f"{lat}",f"{lon}"], zoom_start = 15)
    return map

In [61]:
#12

In [62]:
def new_column_in_df():
    company = ['Company 1', 'Company 2', 'Company 3', 'Company 4','Company 5']
    df2 = df.copy()
    df2['Company'] = company
    return df2

In [ ]:
#13

In [63]:
def markers_for_df(df,map):

    for index, row in df.iterrows():

        #1. MARKER without icon
        location = {"location": [row["latitude"], row["longitude"]], "tooltip": row["Company"]}

        #2. Icon      
        icon = Icon (
                color="red",
                opacity = 0.6,
                prefix = "fa",
                icon="briefcase",
                icon_color = "black"
            )

        #3. Marker
        new_marker = Marker(**location, icon = icon, radius = 2)

        #4. Add the Marker
        new_marker.add_to(f"{map}")
        return map

In [64]:
#14

In [69]:
def markers_for_df_with_conditions(df,map):
        for index, row in df.iterrows():
            

            #1. MARKER without icon
            venue = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}

            #2. Icon
            if row["category"] == "airport":        
                icon = Icon (
                    color="black",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="plane",
                    icon_color = "white"
                )
            elif row["category"] == "starbucks":
                icon = Icon (
                    color="green",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="coffee",
                    icon_color = "black"
                )
            elif row["category"] == "vegan":
                icon = Icon (
                    color="pink",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="leaf",
                    icon_color = "black"
                )
            elif row["category"] == "petcare":
                icon = Icon (
                    color="gray",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="paw",
                    icon_color = "black"
                )     
            else:
                icon = Icon (
                    color="orange",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="futbol-o",
                    icon_color = "black",
                    icon_size=(14, 14)
                )
            #3. Marker
            new_marker = Marker(**venue, icon = icon, radius = 2)

            #4. Add the Marker
            new_marker.add_to(map)
            return map


In [71]:
#15

In [70]:
def haversine(coord1, coord2):

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers
    #return meters
    
    #meters = round(meters, 3)
    km = round(km, 1)
    return km
    
    #print(f"Distance: {meters} m")
    #print(f"Distance: {km} kilometres")‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍

In [74]:
#16

In [49]:
def list_with_distances_to_a_coordinate(df):
    distance_list = []

    for item, rows in nyc_venues3.iterrows():
            coords_1 = (40.755716,-73.979247)
            coords_2 = (df["lat"][item],df["lon"][item])
            distance_trial = (geopy.distance.great_circle(coords_1, coords_2).km)
            final_distance_trial = round(distance_trial,1)
            final_distance_trial
            distance_list_2.append(final_distance_trial)

    return distance_list

In [78]:
#17

In [77]:
def dataframe_with_distance_column_added(df):
    df2 = df.copy()
    df2['distance(km)'] = distance_list
    return df2

In [79]:
#18

In [80]:
def calculating_average_distance(df):
    mean_distances = df.groupby("category")["distance(km)"].mean()
    return mean_distances

In [84]:
#19

In [81]:
def type_point (list_):
    return {"type": "Point", "coordinates": list_}

In [85]:
#20

In [83]:
def find_nyc_area (list_,database,collection):
    client = MongoClient("localhost:27017")
    db = client.get_database(f"{database}")
    collection = db.get_collection(f"{collection}")
    
    type_point_converted = type_point(list_)

    proy_ = {"name":1, "_id": 0}
    return collection.find_one({"geometry":
                        {"$geoIntersects": 
                            {"$geometry": type_point_converted}}}, proy_)

# Foursquare API

In [87]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [96]:
#21

In [90]:
def get_airports_from_foursquare (category_id, location, limit=50):
    if load_dotenv():
        token_fsq = os.getenv("API_Key")
        ll = f"{location[1]}%2C{location[0]}"
        url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&radius=25000"

        headers = {
            "accept": "application/json",
            "Authorization": token_fsq,
        }

        response = requests.get(url, headers=headers).json()

        return response
    else:
        print("There is no possible to run this function,you might be missing any required parameter")

In [ ]:
#22

In [50]:
def list_with_airports():
    def get_airports_from_foursquare (category_id, location, limit=50):
        if load_dotenv():
            token_fsq = os.getenv("API_Key")
            ll = f"{location[1]}%2C{location[0]}"
            url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&radius=25000"

            headers = {
                "accept": "application/json",
                "Authorization": token_fsq,
            }

            response = requests.get(url, headers=headers).json()

            return response
    
    new_list = []
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]

        type_ = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        new_list.append({"name":name, "lat":lat, "lon":lon, "type":type_})

    return new_list

In [ ]:
#23

In [91]:
def get_starbucks_from_foursquare (query, location, limit=50):
    if load_dotenv():
        token_fsq = os.getenv("API_Key")   
        ll = f"{location[1]}%2C{location[0]}"
        url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&limit={str(limit)}&&radius=1000"

        headers = {
            "accept": "application/json",
            "Authorization": token_fsq,
        }

        response = requests.get(url, headers=headers).json()

        return response

    else:
        print("There is no possible to run this function,you might be missing any required parameter")

In [ ]:
#24

In [51]:
def list_with_starbucks():

    def get_starbucks_from_foursquare (query, location, limit=50):
        if load_dotenv():
            token_fsq = os.getenv("API_Key")   
            ll = f"{location[1]}%2C{location[0]}"
            url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&limit={str(limit)}&&radius=1000"

            headers = {
                "accept": "application/json",
                "Authorization": token_fsq,
            }

            response = requests.get(url, headers=headers).json()

            return response

        else:
            print("There is no possible to run this function,you might be missing any required parameter")




    starbucks_list = []
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]

        type_ = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        starbucks_list.append({"name":name, "lat":lat, "lon":lon, "type":type_})

    return starbucks_list

In [ ]:
#25

In [94]:
def get_vegan_rest_from_foursquare (category_id, location, limit=50):
    if load_dotenv():
        token_fsq = os.getenv("API_Key")  
       
        ll = f"{location[1]}%2C{location[0]}"
        url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&&radius=3000"

        headers = {
            "accept": "application/json",
            "Authorization": token_fsq,
        }

        response = requests.get(url, headers=headers).json()
    
        return response
    else:
        print("There is no possible to run this function,you might be missing any required parameter")

In [ ]:
#26

In [52]:
def list_of_vegan_restaurants():
    def get_vegan_rest_from_foursquare (category_id, location, limit=50):
        if load_dotenv():
            token_fsq = os.getenv("API_Key")  

            ll = f"{location[1]}%2C{location[0]}"
            url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&&radius=3000"

            headers = {
                "accept": "application/json",
                "Authorization": token_fsq,
            }

            response = requests.get(url, headers=headers).json()

            return response
        else:
            print("There is no possible to run this function,you might be missing any required parameter")
    

    vegan_list = []
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]

        type_ = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        vegan_list.append({"name":name, "lat":lat, "lon":lon, "type":type_})

    vegan_list

In [ ]:
#27

In [97]:
def get_pets_from_foursquare (category_id, location, limit=50):
    if load_dotenv():
        token_fsq = os.getenv("API_Key")    
        ll = f"{location[1]}%2C{location[0]}"
        url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&&radius=3000"

        headers = {
            "accept": "application/json",
            "Authorization": token_fsq,
        }

        response = requests.get(url, headers=headers).json()

        return response
    else:
        print("There is no possible to run this function,you might be missing any required parameter")        

In [ ]:
#28

In [53]:
def list_of_pet_grooming_places():
    def get_pets_from_foursquare (category_id, location, limit=50):
        if load_dotenv():
            token_fsq = os.getenv("API_Key")    
            ll = f"{location[1]}%2C{location[0]}"
            url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&&radius=3000"

            headers = {
                "accept": "application/json",
                "Authorization": token_fsq,
            }

            response = requests.get(url, headers=headers).json()

            return response
        else:
            print("There is no possible to run this function,you might be missing any required parameter")      
    

    pet_list = []
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]

        type_ = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        pet_list.append({"name":name, "lat":lat, "lon":lon, "type":type_})

    return pet_list

In [ ]:
#29

In [101]:
def get_stadiums_from_foursquare (category_id, location, limit=20):
    if load_dotenv():
        token_fsq = os.getenv("API_Key") 
       
        ll = f"{location[1]}%2C{location[0]}"
        url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&&radius=10000"

        headers = {
            "accept": "application/json",
            "Authorization": token_fsq,
        }

        response = requests.get(url, headers=headers).json()

        return response
    else:
        print("There is no possible to run this function,you might be missing any required parameter")  

In [ ]:
#30

In [54]:
def list_of_stadiums():
    def get_stadiums_from_foursquare (category_id, location, limit=20):
        if load_dotenv():
            token_fsq = os.getenv("API_Key") 

            ll = f"{location[1]}%2C{location[0]}"
            url = f"https://api.foursquare.com/v3/places/search?categories={category_id}&ll={ll}&limit={str(limit)}&&radius=10000"

            headers = {
                "accept": "application/json",
                "Authorization": token_fsq,
            }

            response = requests.get(url, headers=headers).json()

            return response
        else:
            print("There is no possible to run this function,you might be missing any required parameter")     
    

    stadium_list = []
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]

        type_ = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        stadium_list.append({"name":name, "lat":lat, "lon":lon, "type":type_})

    return stadium_list

In [ ]:
#31

In [55]:
def geometry_of_lists(list):
    df = pd.DataFrame(list)
    df2 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]))
    return df2

In [ ]:
#32

In [100]:
def map_with_venues(df,category):
    the_map = Map(Layer(df, "color:black", popup_hover=[popup_element("name", f"{category} places")]))
    return the_map

# Web Scraping

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np

In [1]:
#33

In [2]:
url_ny = "https://www.squarefoot.com/office-space/m/ny/new-york/635d40e4-ee2c-4144-8c95-040714888811?neighborhoods=88d9e668-b2fa-4ebe-a54e-488328f56823%2C487bd69c-8f9d-4441-895b-29d1d40afc17%2C82351707-ced7-4c44-aab6-f39347566197%2C5dc352b4-5a83-4b08-b1d7-bdd50fb85e79%2Ccde1e681-bff5-4c40-b96a-f5ba67354cfc%2Ce6657556-6c44-47e8-b825-f61d1a35896e%2C78edec76-bc46-4b67-8401-6697469f7fbd%2Cac7231b4-c3eb-4141-ad47-6799653ee1f2%2C94e3c05e-07b1-4ec2-9ce6-52de63e49ab0%2Cd6a095c6-cb75-4667-9b78-7ca8528498ee%2C7d2cfc8c-a753-449f-bb19-7054112af57e%2Cf5aa7229-00b5-42d8-9442-55f884ae1f3b&minOccupancy=86&maxOccupancy=120&activeSizeFilter=SEATS&groupByBuilding=false&page="

In [56]:
def getting_all_info_web_scraping(url):

    places_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        name_offering = soup.find_all("p", attrs={"class":"style_title__NcI0n"})

        for i in range(len(name_offering)):
            name_offering[i].getText()
            places_list.append(name_offering)
    
    return places_list[1]

    pricing_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        price_offering = soup.find_all("p", attrs={"class":"style_pricing__TKbkD"})

        for i in range(len(price_offering)):
            price_offering[i].getText()
            pricing_list.append(price_offering)

    return pricing_list[1]

    size_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        size_offering = soup.find_all("span", attrs={"class":"style_squarefeet__q7bHB"})

        for i in range(len(size_offering)):
            size_offering[i].getText()
            size_list.append(size_offering)
    return size_list[1]

    occupancy_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        occ_offering = soup.find_all("span", attrs={"class":"style_maxOccupancy__uKsmM"})

        for i in range(len(occ_offering)):
            occ_offering[i].getText()
            occupancy_list.append(occ_offering)


    return occupancy_list         

In [ ]:
#34

In [40]:
def address_info(url):
    places_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        name_offering = soup.find_all("p", attrs={"class":"style_title__NcI0n"})

        for i in range(len(name_offering)):
            name_offering[i].getText()
            places_list.append(name_offering)

    
    empty_list_places = []

    for i in places_list:
        for item in i:
            empty_list_places.append(item)
 


    places_list2 = []

    for i in empty_list_places:
        only_address = i.getText()
        places_list2.append(only_address)
    return places_list2
    

In [ ]:
#35

In [39]:
def pricing_info(url):
    
    pricing_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        price_offering = soup.find_all("p", attrs={"class":"style_pricing__TKbkD"})

        for i in range(len(price_offering)):
            price_offering[i].getText()
            pricing_list.append(price_offering)

    empty_list_price = []

    for i in pricing_list:
        for item in i:
            empty_list_price.append(item)

    pricing_list2 = []

    for i in empty_list_price:
        only_prices = i.getText().strip()
        if "$" in only_prices:
            pricing_list2.append(only_prices)
        else:
            pricing_list2.append("To ask")

    pricing_list3 = []

    for i in pricing_list2:
        pricing_list3.append(i[:7])

    pricing_list4 = []

    for i in pricing_list3:
        if "$":
            pricing_list4.append(i.replace("$","").replace(",",""))
        else:
            pricing_list4.append
    return pricing_list4
    

In [ ]:
#36

In [38]:
def site_size_info(url):
    size_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        size_offering = soup.find_all("span", attrs={"class":"style_squarefeet__q7bHB"})

        for i in range(len(size_offering)):
            size_offering[i].getText()
            size_list.append(size_offering)

    empty_list_size = []

    for i in size_list:
        for item in i:
            empty_list_size.append(item)

    size_list2 = []

    for i in empty_list_size:
        only_sizes = i.getText()
        size_list2.append(only_sizes[:6])
    return size_list2
    

In [ ]:
#37

In [37]:
def site_max_occupancy(url):
    occupancy_list = []

    for page in range(1,9):
        html = requests.get(f"{url}" + str(page))
        soup = BeautifulSoup(html.content, "html.parser")
        occ_offering = soup.find_all("span", attrs={"class":"style_maxOccupancy__uKsmM"})

        for i in range(len(occ_offering)):
            occ_offering[i].getText()
            occupancy_list.append(occ_offering)

    empty_list_occ = []

    for i in occupancy_list:
        for item in i:
            empty_list_occ.append(item)

    occupancy_list2 = []

    for i in empty_list_occ:
        only_occ = i.getText()
        occupancy_list2.append(only_occ[1:])
    return occupancy_list2